## In this notebook we run thought the data aquisitin pipeline to get derive results discussed in Samuel Heczko's masters thesis

In [7]:
import glob
import os
import connectome
import nibabel as nib
import pandas as pd
import regresson

import sys; sys.path
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.metrics import explained_variance_score, r2_score
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
import imp


path = 'data/' ##cahnge for cluster




/tmp/ipykernel_91172/1430019377.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


### 1. Get the connectomes from preregistered fmri images
In the thesis we largely conducted analysis on thethe Amsterdam Open MRI Collection (AOMIC). We used special preprocessed files and the data proveided by the AOIMIC people. We derive thea analysis in __ steps. All steps are done to all 900 subjects. We loop following proceedure over all atalases considererd.
1. Calcluate the time series data for each brain region for each atlas (extract_connectomes.py)
2. Compute the connectome of for each subject
3. For each atlas save the upper triangle of the connectivity matrix along the information which brain regions are connected in this connection.


Let's start with setting up the parameters

In [3]:
##defime global parameters
cluster = False
correlation_measure = 'correlation' #can be also tangent, partial
res_n = 1
res = f'{res_n}x{res_n}x{res_n}'
correlation_measure='correlation' ##set for calculation of the brain connectome, choose from 'correlation', 'tangent', 'partial' as implemented by nilearn
#templateICBM = '/kyb/agks/sheczko/Downloads/MastersThesis/code/data/templates/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c.nii' ##use the ICBM T1 template


##add the data and names of thigs
if cluster:
    imgs_paths = glob.glob(path + 'func_images/AOMIC/prep_nifti/*.nii') ##load up a subset of the subejct images

else:
    imgs_paths = glob.glob(path + 'func_images/AOMIC/prep_nifti/*000*.nii') ##load up all the subejct images
subjects_idxs = []
for s_n in imgs_paths:
    subjects_idxs.append(s_n.split('_')[-1].split('.')[0]) #split the path to extract only the number of the subject from path



atlases = glob.glob(path + '/atlases/lawrance2021/label/Human/ICBM/*.nii.gz') ##get the atlases
anatomical_labels = glob.glob(path + '/atlases/lawrance2021/label/Human/Anatomical-labels-csv/*.csv') #get the anatomical labels (where available)
anatomical_label_names = []
for a_l in anatomical_labels:
    anatomical_label_names.append(a_l.split('/')[-1].split('.')[0]) #split the path to extract only the name of the atlas


##loop over atlases

Contiune with looping over the list of atlases and saving a csv wiht connectivity for all subejcts.

In [3]:
for atlas_path in atlases: ##loop over atlases
    atlas_name = (atlas_path.split('/')[-1].split('.')[0].split('_')[0]) #split the atlas path so we
    print(atlas_name)
    al_p = (any(n == atlas_name for n in anatomical_label_names)) ##find wheter we have the anatomical labellings for this atlas

    if al_p:
        anatomic_path = path + f'/atlases/lawrance2021/label/Human/Anatomical-labels-csv/' + atlas + '.csv'
        ana_labels = pd.read_csv(anatomic_path,names=colnames, header=None)
    else:
        ana_labels = None


    time_series = connectome.calculate_time_series(atlas_path = atlas_path,imgs_paths=imgs_paths)     ##get the time series of the from all subjects, using the atlas defined

    correlation_matrices, _ =  connectome.connectome(time_series = time_series,correlation_measure='correlation') #get the connectivity matrices

    df_ = connectome.save_connectomes_df(correlation_matrices,anatomical_label_presence = al_p, anatomic_labels = ana_labels,path_to_save = path + 'results/', atlas_name = atlas_name, n_subjects = correlation_matrices.shape[0], correlation_measure = correlation_measure,subject_ixds = subjects_idxs)
    


DS00108
DS06481


KeyboardInterrupt: 

# find the regressons using the features extracted earlier
This part of the code is also regression_launcher.py
Now that we have extracted the features of our fMRI data, we are ready to apply the regression models to calcualte the predictability of our target variables. We subdivide this task into three steps:
1. loading and formatting our connectivity data
2. loading and formattin our target variables
3. fitting the desired model on the defined x and y

This process needs to be repeated over all the atlases and other variables we have been looping over. This allows us to asses the question which steps of the pipline architecture have the largest effect on the prediction accuracy.



In [23]:
imp.reload(regresson)


##let's find the paths to the the csv connectivity data we ahve
data_paths = glob.glob(path + '/results/connectomes/*.csv')

##load up regresors
regressors_df =  regresson.load_regressors(path + 'func_images/AOMIC/regressors/*.txt')
##choose the target variables, take as np arrays
GCA = regressors_df.regressor_iq.values
bmi = regressors_df.regressor_bmi.values

 #concatenate cognitive metrics into single variable
cognition = ['GCA','bmi']
#cognition = ['PMAT Correct', 'PMAT Response Time']
cog_metric = np.transpose(np.asarray([GCA, bmi]))

#set the number of permutations you want to perform
perm = 5
    #set the number of cross-validation loops you want to perform
cv_loops = 5
#set the number of folds you want in the inner and outer folds of the nested cross-validation
k = 3
#set the proportion of data you want in your training set
train_size = .8
#set the number of variable you want to predict to be the number of variables stored in the cognition variablse
n_cog = np.size(cognition)
#set regression model type
regr = Ridge(fit_intercept = True, max_iter=1000000)
#set hyperparameter grid space you want to search through for the model
alphas = np.linspace(100, 1000, num=10, endpoint=True, dtype=None, axis=0)
#set y to be the cognitive metrics you want to predict. They are the same for every atlas (each subject has behavioural score regradless of parcellation)
Y = cog_metric


for data_path_i, data_path in enumerate(data_paths): ##loop over atlases
    
    current_atlas = data_paths[data_path_i+1].split('/')[-1].split('-')[-1].split('.')[0]
    print(f'current atlas: ' + current_atlas)

    fc = regresson.load_data(data_paths[data_path_i+1]) ##set the imput variable to the current atlas connectome, gives subjects x features matrix

    #set x data to be the input variable you want to use
    #ie fc, sc, or hc
    #cut the negative values from the correlations
    X = fc
    X[X<0] = 0


    #set the number of features 
    n_feat = X.shape[1]
    r2, preds, var, corr, featimp, cogtest,opt_alphas = regresson.regression(X = X, Y = Y, perm = perm, cv_loops = cv_loops, k = k, train_size = 0.8, n_cog = n_cog, regr = regr, alphas = alphas,n_feat = n_feat,cognition = cognition)
    
    ##save data:
    result_df = pd.DataFrame() ## empty dataframe, filled with colums where each row is one permutation
    preds_df = pd.DataFrame()

    for cog_i, cog_names in enumerate(cognition):
            result_df[f'{cog_names}' + '_r2'] = r2[:,cog_i] #fill in the r2 values
            result_df[f'{cog_names}' + '_var'] = var[:,cog_i] #fill in explained variation values
            result_df[f'{cog_names}' + '_opt_alphas'] = opt_alphas[:,cog_i] #fill in optimal alpha values
            for perm_n in range(preds.shape[0]): 
                preds_df[f'{cog_names}_perm_{perm_n + 1}_preds'] = preds[perm_n,cog_i,:] #fill in preditions
                preds_df[f'{cog_names}_perm_{perm_n + 1}_real'] = cogtest[perm_n,cog_i,:] #fill in real_values
    result_df.to_csv(path + f'results/ridge_regression/ridge_results_atlas-{current_atlas}.csv')
    preds_df.to_csv(path + f'results/ridge_regression/ridge_preds_atlas-{current_atlas}.csv')



current atlas: AAL
Permutation 1
Cognition: GCA
Training Models


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.9s finished


1/5 Complete


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.9s finished


2/5 Complete


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.9s finished


4/5 Complete


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.9s finished


5/5 Complete
Testing Models


NameError: name 'reg' is not defined

In [27]:
##save data:
result_df = pd.DataFrame() ## empty dataframe, filled with colums where each row is one permutation
preds_df = pd.DataFrame()
for cog_i, cog_names in enumerate(cognition):
        result_df[f'{cog_names}' + '_r2'] = r2[:,cog_i]
        result_df[f'{cog_names}' + '_var'] = var[:,cog_i]
        result_df[f'{cog_names}' + '_opt_alphas'] = opt_alphas[:,cog_i]
        for perm_n in range(preds.shape[0]):
            preds_df[f'{cog_names}_perm_{perm_n + 1}_preds'] = preds[perm_n,cog_i,:]
            preds_df[f'{cog_names}_perm_{perm_n + 1}_real'] = cogtest[perm_n,cog_i,:]
result_df.to_csv(path + f'results/ridge_regression/ridge_results_atlas-{current_atlas}.csv')
preds_df.to_csv(path + f'results/ridge_regression/ridge_preds_atlas-{current_atlas}.csv')


In [19]:
result_df = pd.DataFrame()
for cog_i, cog_names in enumerate(cognition):
        result_df[f'{cog_names}' + '_r2'] = r2[:,cog_i]
        result_df[f'{cog_names}' + '_var'] = var[:,cog_i]

In [16]:
r2 = np.random.random((5,2))
var = np.random.random((5,2))

In [17]:
print(r2)
print(var)

[[0.35293049 0.9468528 ]
 [0.83402726 0.93571856]
 [0.2929712  0.62324246]
 [0.49220652 0.19135977]
 [0.44216661 0.06799257]]
[[0.1901317  0.42403592]
 [0.23709415 0.02885403]
 [0.42005898 0.70998366]
 [0.55858748 0.19009908]
 [0.56388484 0.82895028]]


In [21]:
result_df

,GCA_r2,GCA_var,bmi_r2,bmi_var
0,0.352930,0.190132,0.946853,0.424036
1,0.834027,0.237094,0.935719,0.028854
2,0.292971,0.420059,0.623242,0.709984
3,0.492207,0.558587,0.191360,0.190099
4,0.442167,0.563885,0.067993,0.828950


In [ ]:
|